# 隐马尔科夫模型Hidden Markov Model HMM

 ## 暴力求解
 1. 生成所有的隐藏状态序列 I
 2. 对每个隐藏状态序列， 计算观测序列出现的概率
 
对于含有 N 个隐藏状态的hmm模型，设生成的序列长度为 T，那么共可生成$N^T$中隐藏状态序列，计算概率需要遍历隐藏序列 T，故时间复杂度为$O(TN^T)$

In [49]:
def generate_hidden_sequences(N,T):
    c = []
    def backtracking(r):
        if len(r) == T:
            c.append([i for i in r])
        else:
            for i in range(N):
                r.append(i)
                backtracking(r)
                r.pop()
    backtracking([])
    return c 
c = generate_hidden_sequences(3,3)
print(len(c))
print(c)

27
[[0, 0, 0], [0, 0, 1], [0, 0, 2], [0, 1, 0], [0, 1, 1], [0, 1, 2], [0, 2, 0], [0, 2, 1], [0, 2, 2], [1, 0, 0], [1, 0, 1], [1, 0, 2], [1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 2, 0], [1, 2, 1], [1, 2, 2], [2, 0, 0], [2, 0, 1], [2, 0, 2], [2, 1, 0], [2, 1, 1], [2, 1, 2], [2, 2, 0], [2, 2, 1], [2, 2, 2]]


In [39]:
from pprint import pprint as pp
class HMM():
    def __init__(self):
        super().__init__()
        self.p = [0.2,0.4,0.4]
        self.A = [[0.5,.2,.3],[.3,.5,.2],[.2,.3,.5]]
        self.B = [[.5,.5],[.4,.6],[.7,.3]]
    def print_hmm(self):
        pp('-----------')
        pp(self.p)
        pp(self.A)
        pp(self.B)
        pp('-----------')
h = HMM()
h.print_hmm()

'-----------'
[0.2, 0.4, 0.4]
[[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
[[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
'-----------'


In [50]:
t = [0,1,0]
print(c)
def prob():
    p = [] 
    for i in c:
        _p = h.p[i[0]] * h.B[i[0]][t[0]]
        for k in range(1,len(i)):
            _p *= h.A[i[k-1]][i[k]] * h.B[i[k]][t[k]]
        p.append(_p)
    return p
p = prob()
print(p)
print(sum(p))
        
        

[[0, 0, 0], [0, 0, 1], [0, 0, 2], [0, 1, 0], [0, 1, 1], [0, 1, 2], [0, 2, 0], [0, 2, 1], [0, 2, 2], [1, 0, 0], [1, 0, 1], [1, 0, 2], [1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 2, 0], [1, 2, 1], [1, 2, 2], [2, 0, 0], [2, 0, 1], [2, 0, 2], [2, 1, 0], [2, 1, 1], [2, 1, 2], [2, 2, 0], [2, 2, 1], [2, 2, 2]]
[0.00625, 0.0020000000000000005, 0.00525, 0.0018, 0.0024000000000000002, 0.0016799999999999999, 0.0009, 0.0010799999999999998, 0.0031499999999999996, 0.006000000000000001, 0.0019200000000000007, 0.005040000000000001, 0.007200000000000001, 0.009600000000000003, 0.00672, 0.0009600000000000001, 0.001152, 0.00336, 0.006999999999999999, 0.0022400000000000002, 0.005879999999999999, 0.007559999999999999, 0.010079999999999999, 0.007055999999999998, 0.0042, 0.005039999999999999, 0.014699999999999998]
0.13021800000000003


# 使用前向算法求概率
上述暴力求解当隐藏状态$N$很大时,时间复杂度很高.

通过定义一个隐藏状态的前向概率,可以使用动态规划递推的方式求解出$P(O|\lambda)$

具体的,前向概率定义为$\alpha_t=P(o_1,o_2,...,o_t,i_t=q_i|\lambda)$. 也即在观测序列为$(o_1,o_2,...,o_t)$,并且隐藏状态为$i_t = q_i$时的概率.

可以由$\alpha_t$推断出$\alpha_{t+1}$.

由于 hmm 认为当前状态只受前一状态影响,并且当前观测只受当前隐藏状态影响, 那么可以推出:
$$
\alpha_{t+1}(i) = [\sum_{j=1}^N\alpha_t(j)a_{ji}]b_i(o_{t+1})
$$
翻译成白话文就是: 
> 已有之前所有状态的前向概率,计算到达状态 i 的前向概率: 由于每个隐藏状态均可到达隐藏状态 i,因此需要遍历所有可能的隐藏状态到 i 的概率,当遍历完成后,得到的概率就是观测序列为$o_1,0_2,...,o_t$,隐藏状态为 i 的概率,再乘以由状态 i 转移到观测$o_{t+1}$的概率$b_i(o_{t+1})$,即可得到$\alpha_{t+1}(i)$乘以观测转移概率即可得到定义的前向概率.

可以看到,上述的计算是完全依赖于 hmm 的两个假设的.

## 时间复杂度
待处理的序列长度为$T$, 处理每个$o_t$时需要时间$O(N^2)$, 所以最后的时间复杂度为:
$$
O(N^2T)
$$

In [53]:
def forward_algo(sequence):
    # lens of hidden state
    N = len(h.A)
    # dp array. actually only one vector is enough since t+1 only need t.
    forward_p = []
    
    # first calculate the initial forward prob.
    p = []
    for i in range(N):
        p.append(h.p[i] * h.B[i][sequence[0]])
    forward_p.append(p)
    # then dp: 2,3...
    for i in range(1,len(sequence)): # O(T)
        # i is the hidden state, and we need calculate it's forward_p according to forward_p[-1]
        a = forward_p[-1]
        p = []
        for k in range(N): # O(N**2)
            p.append(sum([a[j] * h.A[j][k] for j in range(N)]) * h.B[k][sequence[i]])
        forward_p.append(p)
    return sum(forward_p[-1])

forward_algo(t)

0.130218